# EVALUACIÓN DE IMPACTO DEL PROGRAMA PTA/FI 3.0
## Metodología AIPW/DR con Wild Cluster Bootstrap

---

### Información del Análisis
- **Fecha de análisis**: Diciembre 2025
- **Metodología**: Augmented Inverse Propensity Weighting / Doubly Robust (AIPW/DR)
- **Inferencia**: Wild Cluster Bootstrap con pesos Rademacher
- **Unidad de clustering**: Escuela (código DANE)
- **Outcomes**: EGRA (lectura) y EGMA (matemáticas)

---

# 1. MARCO METODOLÓGICO

## 1.1 Fundamentos del Estimador AIPW/DR

El estimador **Augmented Inverse Propensity Weighting (AIPW)**, también conocido como **Doubly Robust (DR)**, combina dos enfoques fundamentales de inferencia causal:

### Componente 1: Inverse Propensity Weighting (IPW)
Pondera las observaciones por el inverso de la probabilidad de recibir el tratamiento:

$$\hat{\tau}_{IPW} = \frac{1}{n}\sum_{i=1}^{n}\left[\frac{D_i Y_i}{e(X_i)} - \frac{(1-D_i)Y_i}{1-e(X_i)}\right]$$

### Componente 2: Regression Adjustment (RA)
Modela directamente los resultados potenciales:

$$\hat{\tau}_{RA} = \frac{1}{n}\sum_{i=1}^{n}\left[\hat{\mu}_1(X_i) - \hat{\mu}_0(X_i)\right]$$

### Estimador AIPW/DR Combinado

$$\hat{\tau}_{AIPW} = \frac{1}{n}\sum_{i=1}^{n}\left[\hat{\mu}_1(X_i) - \hat{\mu}_0(X_i) + \frac{D_i(Y_i - \hat{\mu}_1(X_i))}{e(X_i)} - \frac{(1-D_i)(Y_i - \hat{\mu}_0(X_i))}{1-e(X_i)}\right]$$

Donde:
- $D_i$: Indicador de tratamiento (1=tratado, 0=control)
- $Y_i$: Resultado observado (puntaje EGRA/EGMA)
- $e(X_i)$: Propensity Score (probabilidad de tratamiento)
- $\hat{\mu}_1(X_i)$, $\hat{\mu}_0(X_i)$: Resultados esperados predichos

## 1.2 Propiedad de Doble Robustez

El estimador AIPW es **doblemente robusto**: produce estimaciones consistentes si:
1. El modelo de propensión $e(X)$ está correctamente especificado, **O**
2. Los modelos de resultado $\mu_1(X)$, $\mu_0(X)$ están correctamente especificados

## 1.3 Wild Cluster Bootstrap

Dado que el tratamiento se asigna a nivel de **escuela** (cluster), usamos **Wild Cluster Bootstrap** con pesos Rademacher:

1. Para cada iteración bootstrap $b = 1, ..., B$:
   - Generar pesos Rademacher $w_s \in \{-1, +1\}$ para cada escuela $s$
   - Calcular efectos individuales perturbados
   - Obtener ATE bootstrap

2. El error estándar es la desviación estándar de la distribución bootstrap

3. Los intervalos de confianza se obtienen de los percentiles 2.5 y 97.5

# 2. CONFIGURACIÓN DEL ENTORNO

In [ ]:
# ==============================================================================
# IMPORTACIÓN DE LIBRERÍAS
# ==============================================================================

import pandas as pd
import numpy as np
import warnings
import os
from pathlib import Path
from scipy import stats

# Machine Learning
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración global
warnings.filterwarnings('ignore')
np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("="*80)
print("EVALUACIÓN DE IMPACTO PTA/FI 3.0")
print("Metodología AIPW/DR con Wild Cluster Bootstrap")
print("="*80)

In [ ]:
# ==============================================================================
# PARÁMETROS DE CONFIGURACIÓN
# ==============================================================================

BASE_DIR = Path(r"C:\Users\Jhon Narvaez\Downloads")
OUTPUT_DIR = BASE_DIR / "EVALUACION_IMPACTO_FINAL" / "Resultados"
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

CONFIG = {
    'AUROC_MIN': 0.6,
    'AUROC_MAX': 0.9,
    'SMD_MAX': 0.10,
    'CLIP_MIN': 0.01,
    'CLIP_MAX': 0.99,
    'N_BOOTSTRAP': 1000,
    'N_FOLDS': 5,
    'RANDOM_STATE': 42
}

print("\nCONFIGURACIÓN DEL ANÁLISIS:")
print("-" * 40)
for param, valor in CONFIG.items():
    print(f"  {param}: {valor}")

# 3. CARGA DE DATOS

In [ ]:
# ==============================================================================
# CARGA DE LA BASE DE DATOS
# ==============================================================================

ruta_estudiantes = BASE_DIR / "BASES_DEFINITIVAS_AIPW" / "base_estudiante_FINAL.csv"
df_original = pd.read_csv(ruta_estudiantes)

print("\n" + "="*80)
print("BASE DE DATOS CARGADA")
print("="*80)
print(f"\nDimensiones:")
print(f"  - Total registros: {len(df_original):,}")
print(f"  - Total variables: {len(df_original.columns)}")
print(f"\nEstructura de clusters:")
print(f"  - Escuelas únicas: {df_original['codigo_dane'].nunique()}")
print(f"  - Estudiantes tratamiento: {df_original['tratamiento'].sum()}")
print(f"  - Estudiantes control: {(1-df_original['tratamiento']).sum()}")

In [ ]:
# ==============================================================================
# VISUALIZACIÓN DE LA BASE ORIGINAL (PRE-ETL)
# ==============================================================================

print("\n" + "="*80)
print("VISUALIZACIÓN BASE ORIGINAL (ANTES DE ETL)")
print("="*80)

print("\nPrimeras 10 filas:")
display(df_original.head(10))

print("\nEstadísticas descriptivas:")
display(df_original.describe())

print("\nValores faltantes por variable:")
missing = df_original.isnull().sum()
missing_pct = (missing / len(df_original) * 100).round(2)
missing_df = pd.DataFrame({'Faltantes': missing, 'Porcentaje': missing_pct})
display(missing_df[missing_df['Faltantes'] > 0].sort_values('Porcentaje', ascending=False))

In [ ]:
# ==============================================================================
# EXPORTAR BASE DE DATOS PRE-ETL (ORIGINAL)
# ==============================================================================

print("\n" + "="*80)
print("EXPORTANDO BASE DE DATOS PRE-ETL")
print("="*80)

# Crear carpeta para bases de datos
DATOS_DIR = BASE_DIR / "EVALUACION_IMPACTO_FINAL" / "Bases_Datos"
DATOS_DIR.mkdir(exist_ok=True, parents=True)

# Exportar base original
df_original.to_csv(DATOS_DIR / 'BASE_PRE_ETL.csv', index=False, encoding='utf-8-sig')
df_original.to_excel(DATOS_DIR / 'BASE_PRE_ETL.xlsx', index=False)

print(f"\n[OK] Bases PRE-ETL exportadas:")
print(f"  - {DATOS_DIR / 'BASE_PRE_ETL.csv'}")
print(f"  - {DATOS_DIR / 'BASE_PRE_ETL.xlsx'}")
print(f"\nDimensiones: {df_original.shape[0]} filas x {df_original.shape[1]} columnas")

# 4. PROCESO ETL (Extract, Transform, Load)

## 4.1 Construcción de Variables Outcome

### EGRA Total (Lectura)
Promedio de 4 subpruebas: CSL, DPSS, LP, CL

### EGMA Total (Matemáticas)
Promedio de 6 subpruebas: MissNum, CompMatOp, Sumas, Restas, MultDiv, Problemas

In [ ]:
# ==============================================================================
# CONSTRUCCIÓN DE VARIABLES OUTCOME
# ==============================================================================

df = df_original.copy()

EGRA_vars = ['CSL_pct_correctas', 'DPSS_pct_correctas',
             'LP_pct_correctas', 'CL_pct_correctas']

EGMA_vars = ['miss_num_pct_correctas', 'comp_mat_op_pct_correctas',
             'sums_pct_correctas', 'substract_pct_correctas',
             'mult_div_pct_correctas', 'res_problems_pct_correctas']

df['EGRA_Total'] = df[EGRA_vars].mean(axis=1)
df['EGMA_Total'] = df[EGMA_vars].mean(axis=1)

print("\n" + "="*80)
print("CONSTRUCCIÓN DE OUTCOMES")
print("="*80)
for outcome in ['EGRA_Total', 'EGMA_Total']:
    print(f"\n{outcome}:")
    print(f"  Media: {df[outcome].mean():.2f}")
    print(f"  Desv. Est.: {df[outcome].std():.2f}")

## 4.2 Tratamiento de Valores Faltantes

| Variable | % Faltantes | Decisión |
|----------|-------------|----------|
| tei_slope_pre | ~40% | NO imputar - crear indicador binario |
| tei_nivel_pre | <5% | Imputar mediana |
| ruralidad | <5% | Imputar mediana |
| pdet | <5% | Imputar mediana |

In [ ]:
# ==============================================================================
# TRATAMIENTO DE VALORES FALTANTES
# ==============================================================================

print("\n" + "="*80)
print("TRATAMIENTO DE VALORES FALTANTES")
print("="*80)

# TEI Slope - NO imputar, crear indicador
df['tiene_tei_slope'] = df['tei_slope_pre'].notna().astype(int)
print(f"\n1. TEI Slope: Indicador binario creado")
print(f"   Con datos: {df['tiene_tei_slope'].sum()} ({df['tiene_tei_slope'].mean()*100:.1f}%)")

# Variables numéricas
print("\n2. Variables numéricas (imputación con mediana):")
vars_imputar = ['tei_nivel_pre', 'ruralidad', 'pdet', 'matricula_pre', 
                'pct_jornada_completa', 'composicion_grados', 'num_jornadas']

for var in vars_imputar:
    if var in df.columns and df[var].isna().any():
        n_na = df[var].isna().sum()
        mediana = df[var].median()
        df[var] = df[var].fillna(mediana)
        print(f"   {var}: {n_na} imputados con mediana = {mediana:.2f}")

## 4.3 Creación de Variables Dummy

In [ ]:
# ==============================================================================
# CREACIÓN DE VARIABLES DUMMY
# ==============================================================================

print("\n" + "="*80)
print("CREACIÓN DE VARIABLES DUMMY")
print("="*80)

if 'zona_ptafi' in df.columns:
    moda_zona = df['zona_ptafi'].mode()[0] if not df['zona_ptafi'].mode().empty else 'C'
    df['zona_ptafi'] = df['zona_ptafi'].fillna(moda_zona)
    
    zona_dummies = pd.get_dummies(df['zona_ptafi'], prefix='zona', drop_first=True)
    df = pd.concat([df, zona_dummies], axis=1)
    
    print(f"\nDummies de zona creadas: {list(zona_dummies.columns)}")
    for col in zona_dummies.columns:
        print(f"  {col}: {df[col].sum()} ({df[col].mean()*100:.1f}%)")

## 4.4 Definición de Covariables

In [ ]:
# ==============================================================================
# DEFINICIÓN DE COVARIABLES
# ==============================================================================

print("\n" + "="*80)
print("DEFINICIÓN DE COVARIABLES")
print("="*80)

zona_cols = [c for c in df.columns if c.startswith('zona_') and c != 'zona_ptafi']

COVARIABLES = [
    'edad', 'sexo',
    'ruralidad', 'pdet',
    'tei_nivel_pre',
    'matricula_pre',
    'pct_jornada_completa',
    'composicion_grados',
] + zona_cols

COVARIABLES = [c for c in COVARIABLES if c in df.columns]

print(f"\nCovariables seleccionadas ({len(COVARIABLES)}):")
for cov in COVARIABLES:
    print(f"  - {cov}: NaN={df[cov].isna().sum()}")

print("\nVariables EXCLUIDAS (alto SMD):")
print("  - tiene_tei_slope: SMD alto")
print("  - num_jornadas: SMD alto")

In [ ]:
# ==============================================================================
# EXPORTAR BASE DE DATOS POST-ETL (PROCESADA)
# ==============================================================================

print("\n" + "="*80)
print("EXPORTANDO BASE DE DATOS POST-ETL")
print("="*80)

# Seleccionar todas las variables relevantes para exportar
cols_exportar = ['codigo_dane', 'tratamiento', 'edad', 'sexo']

# Agregar outcomes
cols_exportar += ['EGRA_Total', 'EGMA_Total']

# Agregar subpruebas EGRA
cols_exportar += EGRA_vars

# Agregar subpruebas EGMA
cols_exportar += EGMA_vars

# Agregar covariables
cols_exportar += COVARIABLES

# Agregar variables adicionales
cols_adicionales = ['tiene_tei_slope', 'tei_slope_pre', 'zona_ptafi']
cols_exportar += [c for c in cols_adicionales if c in df.columns]

# Agregar dummies de zona
cols_exportar += [c for c in df.columns if c.startswith('zona_') and c not in cols_exportar]

# Eliminar duplicados manteniendo orden
cols_exportar = list(dict.fromkeys(cols_exportar))
cols_exportar = [c for c in cols_exportar if c in df.columns]

# Crear DataFrame de exportación
df_post_etl = df[cols_exportar].copy()

# Exportar
df_post_etl.to_csv(DATOS_DIR / 'BASE_POST_ETL.csv', index=False, encoding='utf-8-sig')
df_post_etl.to_excel(DATOS_DIR / 'BASE_POST_ETL.xlsx', index=False)

print(f"\n[OK] Bases POST-ETL exportadas:")
print(f"  - {DATOS_DIR / 'BASE_POST_ETL.csv'}")
print(f"  - {DATOS_DIR / 'BASE_POST_ETL.xlsx'}")
print(f"\nDimensiones: {df_post_etl.shape[0]} filas x {df_post_etl.shape[1]} columnas")
print(f"\nVariables incluidas ({len(cols_exportar)}):")
for i, col in enumerate(cols_exportar, 1):
    print(f"  {i:2d}. {col}")

In [ ]:
# ==============================================================================
# VISUALIZACIÓN DE LA BASE PROCESADA (POST-ETL)
# ==============================================================================

print("\n" + "="*80)
print("VISUALIZACIÓN BASE PROCESADA (DESPUÉS DE ETL)")
print("="*80)

cols_mostrar = ['codigo_dane', 'tratamiento', 'edad', 'sexo', 'EGRA_Total', 'EGMA_Total']
cols_mostrar = [c for c in cols_mostrar if c in df.columns]

print("\nPrimeras 10 filas (variables principales):")
display(df[cols_mostrar].head(10))

print("\nResumen de covariables:")
display(df[COVARIABLES].describe())

# 5. ANÁLISIS DE VARIANZA

In [ ]:
# ==============================================================================
# ANÁLISIS DE VARIANZA
# ==============================================================================

print("\n" + "="*80)
print("ANÁLISIS DE VARIANZA")
print("="*80)

def calcular_icc(df, outcome, grupo_var='codigo_dane'):
    grupos = df.groupby(grupo_var)[outcome]
    medias_grupo = grupos.mean()
    media_global = df[outcome].mean()
    n_por_grupo = grupos.size()
    var_between = np.sum(n_por_grupo * (medias_grupo - media_global)**2) / (len(df) - 1)
    var_within = grupos.var().mean()
    icc = var_between / (var_between + var_within) if (var_between + var_within) > 0 else 0
    return icc

print("\n1. CORRELACIÓN INTRACLASE (ICC):")
for outcome in ['EGRA_Total', 'EGMA_Total']:
    icc = calcular_icc(df, outcome)
    print(f"   {outcome}: ICC = {icc:.4f} ({icc*100:.1f}% varianza entre escuelas)")

print("\n2. TEST DE LEVENE:")
for outcome in ['EGRA_Total', 'EGMA_Total']:
    trat = df[df['tratamiento'] == 1][outcome].dropna()
    ctrl = df[df['tratamiento'] == 0][outcome].dropna()
    stat, p = stats.levene(trat, ctrl)
    print(f"   {outcome}: W={stat:.3f}, p={p:.4f}")

print("\n3. ANOVA ENTRE ESCUELAS:")
for outcome in ['EGRA_Total', 'EGMA_Total']:
    grupos_data = [g[outcome].dropna().values for _, g in df.groupby('codigo_dane')]
    grupos_data = [g for g in grupos_data if len(g) > 0]
    f_stat, p_val = stats.f_oneway(*grupos_data)
    print(f"   {outcome}: F={f_stat:.3f}, p={p_val:.4f}")

# 6. FUNCIONES AUXILIARES

In [ ]:
# ==============================================================================
# FUNCIONES AUXILIARES
# ==============================================================================

def calcular_smd(df, var, tratamiento_col='tratamiento'):
    """Calcula el Standardized Mean Difference (SMD)"""
    trat = df[df[tratamiento_col] == 1][var].dropna()
    ctrl = df[df[tratamiento_col] == 0][var].dropna()
    if len(trat) == 0 or len(ctrl) == 0:
        return 0
    var_pooled = np.sqrt((trat.var() + ctrl.var()) / 2)
    if var_pooled == 0:
        return 0
    return abs((trat.mean() - ctrl.mean()) / var_pooled)

def calcular_ess(weights):
    """Calcula el Effective Sample Size (ESS)"""
    return (np.sum(weights) ** 2) / np.sum(weights ** 2)

def smd_ponderado(df, var, weights_norm, y):
    """Calcula el SMD después de aplicar pesos IPW"""
    mask_t = y == 1
    mask_c = y == 0
    mean_t = np.average(df[var].values[mask_t], weights=weights_norm[mask_t])
    mean_c = np.average(df[var].values[mask_c], weights=weights_norm[mask_c])
    var_t = np.average((df[var].values[mask_t] - mean_t)**2, weights=weights_norm[mask_t])
    var_c = np.average((df[var].values[mask_c] - mean_c)**2, weights=weights_norm[mask_c])
    var_pooled = np.sqrt((var_t + var_c) / 2)
    return abs((mean_t - mean_c) / var_pooled) if var_pooled > 0 else 0

print("Funciones auxiliares definidas: calcular_smd, calcular_ess, smd_ponderado")

# 7. MODELO DE PROPENSIÓN

In [ ]:
# ==============================================================================
# MODELO DE PROPENSIÓN (Logistic Ridge)
# ==============================================================================

print("\n" + "="*80)
print("MODELO DE PROPENSIÓN")
print("="*80)

X = df[COVARIABLES].values
y = df['tratamiento'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"\nEntrenando Logistic Ridge...")
model = LogisticRegressionCV(
    cv=5, penalty='l2', solver='lbfgs',
    Cs=np.logspace(-4, 2, 20), max_iter=2000,
    random_state=CONFIG['RANDOM_STATE']
)
model.fit(X_scaled, y)

ps = model.predict_proba(X_scaled)[:, 1]
ps = np.clip(ps, CONFIG['CLIP_MIN'], CONFIG['CLIP_MAX'])

df['propensity_score'] = ps

print(f"\nC óptimo: {model.C_[0]:.4f}")
print(f"\nDistribución de Propensity Scores:")
print(f"  Min: {ps.min():.4f}")
print(f"  Mediana: {np.median(ps):.4f}")
print(f"  Max: {ps.max():.4f}")

# 8. DIAGNÓSTICOS DEL MODELO

In [ ]:
# ==============================================================================
# DIAGNÓSTICOS: AUROC Y ESS
# ==============================================================================

print("\n" + "="*80)
print("DIAGNÓSTICOS DEL MODELO")
print("="*80)

auroc = roc_auc_score(y, ps)
print(f"\n1. AUROC = {auroc:.4f}")
if 0.6 <= auroc <= 0.9:
    print(f"   [OK] En rango óptimo [0.6, 0.9]")

# ESS
weights_t = 1 / ps[y == 1]
weights_c = 1 / (1 - ps[y == 0])
weights_t_norm = weights_t / weights_t.sum() * len(weights_t)
weights_c_norm = weights_c / weights_c.sum() * len(weights_c)

ess_t = calcular_ess(weights_t_norm)
ess_c = calcular_ess(weights_c_norm)
n_tratados = (y == 1).sum()
n_control = (y == 0).sum()

print(f"\n2. EFFECTIVE SAMPLE SIZE (ESS):")
print(f"   Tratamiento: {ess_t:.0f}/{n_tratados} ({ess_t/n_tratados*100:.1f}%)")
print(f"   Control: {ess_c:.0f}/{n_control} ({ess_c/n_control*100:.1f}%)")

In [ ]:
# ==============================================================================
# BALANCE DE COVARIABLES (SMD)
# ==============================================================================

print("\n" + "="*80)
print("BALANCE DE COVARIABLES (SMD)")
print("="*80)

# SMD SIN PONDERAR
print(f"\n3. SMD SIN PONDERAR:")
smds_crudo = {}
for var in COVARIABLES:
    smd = calcular_smd(df, var)
    smds_crudo[var] = smd
    estado = "[OK]" if smd < 0.10 else "[!]" if smd < 0.25 else "[X]"
    print(f"   {var:30s}: {smd:.4f} {estado}")

smd_promedio_crudo = np.mean(list(smds_crudo.values()))
print(f"\n   SMD Promedio: {smd_promedio_crudo:.4f}")

# SMD PONDERADO
print(f"\n4. SMD PONDERADO (IPW):")
weights_ipw = np.where(y == 1, 1/ps, 1/(1-ps))
weights_norm = weights_ipw.copy()
weights_norm[y==1] = weights_ipw[y==1] / weights_ipw[y==1].sum() * (y==1).sum()
weights_norm[y==0] = weights_ipw[y==0] / weights_ipw[y==0].sum() * (y==0).sum()

smds_ipw = {}
for var in COVARIABLES:
    smd_w = smd_ponderado(df, var, weights_norm, y)
    smds_ipw[var] = smd_w
    mejora = ((smds_crudo[var] - smd_w) / smds_crudo[var] * 100) if smds_crudo[var] > 0 else 0
    estado = "[OK]" if smd_w < 0.10 else "[!]"
    print(f"   {var:30s}: {smd_w:.4f} {estado} (mejora: {mejora:+.1f}%)")

smd_promedio_ipw = np.mean(list(smds_ipw.values()))
print(f"\n   SMD Promedio (IPW): {smd_promedio_ipw:.4f}")

# 9. LOVE PLOT - VISUALIZACIÓN DEL BALANCE

In [ ]:
# ==============================================================================
# LOVE PLOT
# ==============================================================================

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# SMD Crudo
ax1 = axes[0]
smds_sorted = sorted(smds_crudo.items(), key=lambda x: x[1], reverse=True)
vars_sorted = [x[0] for x in smds_sorted]
vals_sorted = [x[1] for x in smds_sorted]
colors = ['red' if v > 0.25 else 'orange' if v > 0.10 else 'green' for v in vals_sorted]

ax1.barh(vars_sorted, vals_sorted, color=colors, alpha=0.7, edgecolor='black')
ax1.axvline(0.10, color='green', linestyle='--', linewidth=2, label='Meta SMD < 0.10')
ax1.axvline(0.25, color='red', linestyle='--', linewidth=2, label='Umbral SMD = 0.25')
ax1.set_xlabel('SMD')
ax1.set_title('Love Plot - SMD SIN PONDERAR', fontweight='bold')
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3, axis='x')

# SMD Ponderado
ax2 = axes[1]
smds_ipw_sorted = sorted(smds_ipw.items(), key=lambda x: x[1], reverse=True)
vars_ipw = [x[0] for x in smds_ipw_sorted]
vals_ipw = [x[1] for x in smds_ipw_sorted]
colors_ipw = ['red' if v > 0.25 else 'orange' if v > 0.10 else 'green' for v in vals_ipw]

ax2.barh(vars_ipw, vals_ipw, color=colors_ipw, alpha=0.7, edgecolor='black')
ax2.axvline(0.10, color='green', linestyle='--', linewidth=2)
ax2.axvline(0.25, color='red', linestyle='--', linewidth=2)
ax2.set_xlabel('SMD')
ax2.set_title('Love Plot - SMD PONDERADO (IPW)', fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'Love_Plot_SMD.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"\n[OK] Love Plot guardado")

# 10. MODELOS DE RESULTADO (Gradient Boosting con Cross-Fitting)

In [ ]:
# ==============================================================================
# MODELOS DE RESULTADO CON CROSS-FITTING
# ==============================================================================

def entrenar_modelos_resultado(df, outcome_var, covariables, n_folds=5):
    """Entrena modelos de resultado con Cross-Fitting usando GroupKFold"""
    X = df[covariables].values
    y_outcome = df[outcome_var].values
    D = df['tratamiento'].values
    groups = df['codigo_dane'].values
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    mu1_hat = np.zeros(len(df))
    mu0_hat = np.zeros(len(df))
    
    gkf = GroupKFold(n_splits=min(n_folds, len(np.unique(groups))))
    
    for fold, (train_idx, test_idx) in enumerate(gkf.split(X_scaled, y_outcome, groups)):
        train_treated = train_idx[D[train_idx] == 1]
        train_control = train_idx[D[train_idx] == 0]
        
        model_1 = GradientBoostingRegressor(
            n_estimators=100, max_depth=3, learning_rate=0.1,
            min_samples_leaf=5, random_state=CONFIG['RANDOM_STATE']
        )
        model_0 = GradientBoostingRegressor(
            n_estimators=100, max_depth=3, learning_rate=0.1,
            min_samples_leaf=5, random_state=CONFIG['RANDOM_STATE']
        )
        
        if len(train_treated) >= 5:
            model_1.fit(X_scaled[train_treated], y_outcome[train_treated])
            mu1_hat[test_idx] = model_1.predict(X_scaled[test_idx])
        else:
            mu1_hat[test_idx] = y_outcome[D == 1].mean()
        
        if len(train_control) >= 5:
            model_0.fit(X_scaled[train_control], y_outcome[train_control])
            mu0_hat[test_idx] = model_0.predict(X_scaled[test_idx])
        else:
            mu0_hat[test_idx] = y_outcome[D == 0].mean()
    
    return mu1_hat, mu0_hat

print("Función entrenar_modelos_resultado() definida")

# 11. ESTIMADOR AIPW Y WILD CLUSTER BOOTSTRAP

In [ ]:
# ==============================================================================
# ESTIMADOR AIPW Y WILD CLUSTER BOOTSTRAP
# ==============================================================================

def calcular_aipw(Y, D, mu1_hat, mu0_hat, ps, clip_min=0.01, clip_max=0.99):
    """Calcula el estimador AIPW"""
    ps_clipped = np.clip(ps, clip_min, clip_max)
    diff_predicha = mu1_hat - mu0_hat
    corr_tratados = D * (Y - mu1_hat) / ps_clipped
    corr_controles = (1 - D) * (Y - mu0_hat) / (1 - ps_clipped)
    tau_i = diff_predicha + corr_tratados - corr_controles
    ate = np.mean(tau_i)
    return ate, tau_i

def wild_cluster_bootstrap(df, ate_original, tau_i, n_bootstrap=1000):
    """Wild Cluster Bootstrap con pesos Rademacher"""
    escuelas = df['codigo_dane'].unique()
    n_escuelas = len(escuelas)
    escuela_idx = {esc: df['codigo_dane'] == esc for esc in escuelas}
    ate_bootstrap = []
    
    for b in range(n_bootstrap):
        if (b + 1) % 200 == 0:
            print(f"    Bootstrap {b + 1}/{n_bootstrap}...")
        
        rademacher = np.random.choice([1, -1], size=n_escuelas)
        tau_wild = np.zeros(len(df))
        
        for i, esc in enumerate(escuelas):
            mask = escuela_idx[esc]
            tau_wild[mask] = ate_original + rademacher[i] * (tau_i[mask] - ate_original)
        
        ate_bootstrap.append(np.mean(tau_wild))
    
    ate_bootstrap = np.array(ate_bootstrap)
    se = np.std(ate_bootstrap)
    ic_lower = np.percentile(ate_bootstrap, 2.5)
    ic_upper = np.percentile(ate_bootstrap, 97.5)
    t_stat = ate_original / se if se > 0 else 0
    p_value = 2 * (1 - stats.norm.cdf(abs(t_stat)))
    
    return {
        'ate': ate_original, 'se': se,
        'ic_lower': ic_lower, 'ic_upper': ic_upper,
        'p_value': p_value, 't_stat': t_stat,
        'bootstrap_dist': ate_bootstrap
    }

print("Funciones calcular_aipw() y wild_cluster_bootstrap() definidas")

# 12. ESTIMACIÓN FINAL

In [ ]:
# ==============================================================================
# ESTIMACIÓN FINAL AIPW/DR
# ==============================================================================

print("\n" + "="*80)
print("ESTIMACIÓN FINAL AIPW/DR")
print("="*80)

resultados = {}

for outcome_name, outcome_var in [('EGRA Total', 'EGRA_Total'), ('EGMA Total', 'EGMA_Total')]:
    print(f"\n{'='*60}")
    print(f"  {outcome_name}")
    print(f"{'='*60}")
    
    # Diferencia simple
    trat_mean = df[df['tratamiento']==1][outcome_var].mean()
    ctrl_mean = df[df['tratamiento']==0][outcome_var].mean()
    diff_simple = trat_mean - ctrl_mean
    print(f"\n  Diferencia simple: {diff_simple:.2f}")
    
    # Modelos de resultado
    print(f"\n  Entrenando modelos con Cross-Fitting...")
    mu1, mu0 = entrenar_modelos_resultado(df, outcome_var, COVARIABLES)
    
    # AIPW
    Y = df[outcome_var].values
    D = df['tratamiento'].values
    ps_vals = df['propensity_score'].values
    
    ate, tau_i = calcular_aipw(Y, D, mu1, mu0, ps_vals)
    print(f"  ATE AIPW puntual: {ate:.4f}")
    
    # Wild Cluster Bootstrap
    print(f"\n  Wild Cluster Bootstrap (B={CONFIG['N_BOOTSTRAP']})...")
    resultado = wild_cluster_bootstrap(df, ate, tau_i, CONFIG['N_BOOTSTRAP'])
    resultados[outcome_name] = resultado
    
    sig = "***" if resultado['p_value'] < 0.01 else "**" if resultado['p_value'] < 0.05 else "*" if resultado['p_value'] < 0.10 else ""
    print(f"\n  RESULTADO:")
    print(f"    ATE = {resultado['ate']:.4f} {sig}")
    print(f"    SE  = {resultado['se']:.4f}")
    print(f"    IC 95% = [{resultado['ic_lower']:.4f}, {resultado['ic_upper']:.4f}]")
    print(f"    p-valor = {resultado['p_value']:.4f}")

# 13.1 ESTIMACIÓN POR SUBPRUEBAS

Además de los índices totales (EGRA Total y EGMA Total), estimamos el efecto del tratamiento para cada subprueba individual:

### Subpruebas EGRA (Lectura):
1. **CSL**: Conocimiento del Sonido de las Letras
2. **DPSS**: Decodificación de Pseudopalabras Sin Sentido
3. **LP**: Lectura de Palabras
4. **CL**: Comprensión Lectora

### Subpruebas EGMA (Matemáticas):
1. **MissNum**: Números Faltantes
2. **CompMatOp**: Comparación y Operaciones Matemáticas
3. **Sumas**: Adición
4. **Restas**: Sustracción
5. **MultDiv**: Multiplicación y División
6. **Problemas**: Resolución de Problemas

In [ ]:
# ==============================================================================
# ESTIMACIÓN DE SUBPRUEBAS EGRA
# ==============================================================================

print("\n" + "="*80)
print("ESTIMACIÓN DE SUBPRUEBAS EGRA (LECTURA)")
print("="*80)

# Definir subpruebas EGRA con nombres legibles
EGRA_subpruebas = {
    'CSL_pct_correctas': 'Conocimiento Sonidos Letras (CSL)',
    'DPSS_pct_correctas': 'Decodificación Pseudopalabras (DPSS)',
    'LP_pct_correctas': 'Lectura de Palabras (LP)',
    'CL_pct_correctas': 'Comprensión Lectora (CL)'
}

resultados_egra_sub = {}

for var, nombre in EGRA_subpruebas.items():
    print(f"\n  {nombre}...")
    
    # Entrenar modelos
    mu1, mu0 = entrenar_modelos_resultado(df, var, COVARIABLES)
    
    # AIPW
    Y = df[var].values
    D = df['tratamiento'].values
    ps_vals = df['propensity_score'].values
    
    ate, tau_i = calcular_aipw(Y, D, mu1, mu0, ps_vals)
    
    # Bootstrap (menos iteraciones para subpruebas)
    resultado = wild_cluster_bootstrap(df, ate, tau_i, n_bootstrap=500)
    resultados_egra_sub[nombre] = resultado
    resultados_egra_sub[nombre]['variable'] = var
    
    sig = "***" if resultado['p_value'] < 0.01 else "**" if resultado['p_value'] < 0.05 else "*" if resultado['p_value'] < 0.10 else ""
    print(f"    ATE = {resultado['ate']:.2f} (SE={resultado['se']:.2f}, p={resultado['p_value']:.3f}) {sig}")

print("\n" + "-"*60)
print("RESUMEN SUBPRUEBAS EGRA:")
print("-"*60)
print(f"{'Subprueba':<45} {'ATE':>8} {'SE':>8} {'p-valor':>10}")
print("-"*60)
for nombre, res in resultados_egra_sub.items():
    sig = "***" if res['p_value'] < 0.01 else "**" if res['p_value'] < 0.05 else "*" if res['p_value'] < 0.10 else ""
    print(f"{nombre:<45} {res['ate']:>8.2f} {res['se']:>8.2f} {res['p_value']:>10.3f} {sig}")

In [ ]:
# ==============================================================================
# ESTIMACIÓN DE SUBPRUEBAS EGMA
# ==============================================================================

print("\n" + "="*80)
print("ESTIMACIÓN DE SUBPRUEBAS EGMA (MATEMÁTICAS)")
print("="*80)

# Definir subpruebas EGMA con nombres legibles
EGMA_subpruebas = {
    'miss_num_pct_correctas': 'Números Faltantes (MissNum)',
    'comp_mat_op_pct_correctas': 'Comparación y Operaciones (CompMatOp)',
    'sums_pct_correctas': 'Sumas',
    'substract_pct_correctas': 'Restas',
    'mult_div_pct_correctas': 'Multiplicación/División (MultDiv)',
    'res_problems_pct_correctas': 'Resolución de Problemas'
}

resultados_egma_sub = {}

for var, nombre in EGMA_subpruebas.items():
    print(f"\n  {nombre}...")
    
    # Entrenar modelos
    mu1, mu0 = entrenar_modelos_resultado(df, var, COVARIABLES)
    
    # AIPW
    Y = df[var].values
    D = df['tratamiento'].values
    ps_vals = df['propensity_score'].values
    
    ate, tau_i = calcular_aipw(Y, D, mu1, mu0, ps_vals)
    
    # Bootstrap (menos iteraciones para subpruebas)
    resultado = wild_cluster_bootstrap(df, ate, tau_i, n_bootstrap=500)
    resultados_egma_sub[nombre] = resultado
    resultados_egma_sub[nombre]['variable'] = var
    
    sig = "***" if resultado['p_value'] < 0.01 else "**" if resultado['p_value'] < 0.05 else "*" if resultado['p_value'] < 0.10 else ""
    print(f"    ATE = {resultado['ate']:.2f} (SE={resultado['se']:.2f}, p={resultado['p_value']:.3f}) {sig}")

print("\n" + "-"*60)
print("RESUMEN SUBPRUEBAS EGMA:")
print("-"*60)
print(f"{'Subprueba':<45} {'ATE':>8} {'SE':>8} {'p-valor':>10}")
print("-"*60)
for nombre, res in resultados_egma_sub.items():
    sig = "***" if res['p_value'] < 0.01 else "**" if res['p_value'] < 0.05 else "*" if res['p_value'] < 0.10 else ""
    print(f"{nombre:<45} {res['ate']:>8.2f} {res['se']:>8.2f} {res['p_value']:>10.3f} {sig}")

In [ ]:
# ==============================================================================
# EXPORTAR RESULTADOS DE SUBPRUEBAS A CSV Y EXCEL
# ==============================================================================

print("\n" + "="*80)
print("EXPORTANDO RESULTADOS DE SUBPRUEBAS")
print("="*80)

# Combinar todos los resultados de subpruebas
todas_subpruebas = []

# EGRA
for nombre, res in resultados_egra_sub.items():
    todas_subpruebas.append({
        'ATE': res['ate'],
        'SE': res['se'],
        'IC_Lower': res['ic_lower'],
        'IC_Upper': res['ic_upper'],
        'p_value': res['p_value'],
        't_stat': res['t_stat'],
        'n': len(df),
        'n_trat': (df['tratamiento']==1).sum(),
        'n_ctrl': (df['tratamiento']==0).sum(),
        'Variable': res['variable'],
        'Nombre': nombre,
        'Tipo': 'EGRA',
        'Significativo_05': res['p_value'] < 0.05,
        'Significativo_10': res['p_value'] < 0.10
    })

# EGMA
for nombre, res in resultados_egma_sub.items():
    todas_subpruebas.append({
        'ATE': res['ate'],
        'SE': res['se'],
        'IC_Lower': res['ic_lower'],
        'IC_Upper': res['ic_upper'],
        'p_value': res['p_value'],
        't_stat': res['t_stat'],
        'n': len(df),
        'n_trat': (df['tratamiento']==1).sum(),
        'n_ctrl': (df['tratamiento']==0).sum(),
        'Variable': res['variable'],
        'Nombre': nombre,
        'Tipo': 'EGMA',
        'Significativo_05': res['p_value'] < 0.05,
        'Significativo_10': res['p_value'] < 0.10
    })

# Agregar totales
for nombre, res in resultados.items():
    tipo = 'TOTAL'
    var = 'EGRA_Total' if 'EGRA' in nombre else 'EGMA_Total'
    todas_subpruebas.append({
        'ATE': res['ate'],
        'SE': res['se'],
        'IC_Lower': res['ic_lower'],
        'IC_Upper': res['ic_upper'],
        'p_value': res['p_value'],
        't_stat': res['t_stat'],
        'n': len(df),
        'n_trat': (df['tratamiento']==1).sum(),
        'n_ctrl': (df['tratamiento']==0).sum(),
        'Variable': var,
        'Nombre': nombre.replace(' Total', ' Total (Lectura)') if 'EGRA' in nombre else nombre.replace(' Total', ' Total (Matematicas)'),
        'Tipo': tipo,
        'Significativo_05': res['p_value'] < 0.05,
        'Significativo_10': res['p_value'] < 0.10
    })

# Crear DataFrame y exportar
df_subpruebas = pd.DataFrame(todas_subpruebas)
df_subpruebas.to_csv(OUTPUT_DIR / 'resultados_subpruebas.csv', index=False, encoding='utf-8-sig')
df_subpruebas.to_excel(OUTPUT_DIR / 'resultados_subpruebas.xlsx', index=False)

print(f"\n[OK] Archivos exportados:")
print(f"  - {OUTPUT_DIR / 'resultados_subpruebas.csv'}")
print(f"  - {OUTPUT_DIR / 'resultados_subpruebas.xlsx'}")

print(f"\n" + "-"*80)
print("TABLA COMPLETA DE RESULTADOS POR SUBPRUEBA:")
print("-"*80)
display(df_subpruebas[['Nombre', 'Tipo', 'ATE', 'SE', 'IC_Lower', 'IC_Upper', 'p_value', 'Significativo_05']])

In [ ]:
# ==============================================================================
# FOREST PLOT DE SUBPRUEBAS EGRA Y EGMA
# ==============================================================================

fig, axes = plt.subplots(1, 2, figsize=(16, 10))

# ============ FOREST PLOT EGRA ============
ax1 = axes[0]
egra_nombres = list(resultados_egra_sub.keys()) + ['EGRA Total']
egra_ates = [res['ate'] for res in resultados_egra_sub.values()] + [resultados['EGRA Total']['ate']]
egra_lowers = [res['ic_lower'] for res in resultados_egra_sub.values()] + [resultados['EGRA Total']['ic_lower']]
egra_uppers = [res['ic_upper'] for res in resultados_egra_sub.values()] + [resultados['EGRA Total']['ic_upper']]
egra_pvals = [res['p_value'] for res in resultados_egra_sub.values()] + [resultados['EGRA Total']['p_value']]

y_pos = np.arange(len(egra_nombres))
colors_egra = ['steelblue' if p < 0.10 else 'gray' for p in egra_pvals]
colors_egra[-1] = 'darkblue'  # Total en color diferente

for i, (ate, lower, upper, color) in enumerate(zip(egra_ates, egra_lowers, egra_uppers, colors_egra)):
    ax1.errorbar(ate, i, xerr=[[ate-lower], [upper-ate]], fmt='o', color=color, 
                 capsize=5, capthick=2, markersize=10, markeredgecolor='black', linewidth=2)

ax1.axvline(0, color='red', linestyle='--', linewidth=2, label='Sin efecto')
ax1.set_yticks(y_pos)
ax1.set_yticklabels(egra_nombres, fontsize=10)
ax1.set_xlabel('ATE (puntos porcentuales)', fontsize=11)
ax1.set_title('Forest Plot - Subpruebas EGRA (Lectura)\nIC 95% Wild Cluster Bootstrap', fontweight='bold', fontsize=12)
ax1.grid(True, alpha=0.3, axis='x')
ax1.legend(loc='lower right')

# Agregar p-valores
for i, (ate, p) in enumerate(zip(egra_ates, egra_pvals)):
    sig = "***" if p < 0.01 else "**" if p < 0.05 else "*" if p < 0.10 else ""
    ax1.annotate(f'p={p:.3f}{sig}', xy=(ate, i), xytext=(5, 0), 
                 textcoords='offset points', fontsize=9, va='center')

# ============ FOREST PLOT EGMA ============
ax2 = axes[1]
egma_nombres = list(resultados_egma_sub.keys()) + ['EGMA Total']
egma_ates = [res['ate'] for res in resultados_egma_sub.values()] + [resultados['EGMA Total']['ate']]
egma_lowers = [res['ic_lower'] for res in resultados_egma_sub.values()] + [resultados['EGMA Total']['ic_lower']]
egma_uppers = [res['ic_upper'] for res in resultados_egma_sub.values()] + [resultados['EGMA Total']['ic_upper']]
egma_pvals = [res['p_value'] for res in resultados_egma_sub.values()] + [resultados['EGMA Total']['p_value']]

y_pos = np.arange(len(egma_nombres))
colors_egma = ['forestgreen' if p < 0.10 else 'gray' for p in egma_pvals]
colors_egma[-1] = 'darkgreen'  # Total en color diferente

for i, (ate, lower, upper, color) in enumerate(zip(egma_ates, egma_lowers, egma_uppers, colors_egma)):
    ax2.errorbar(ate, i, xerr=[[ate-lower], [upper-ate]], fmt='s', color=color,
                 capsize=5, capthick=2, markersize=10, markeredgecolor='black', linewidth=2)

ax2.axvline(0, color='red', linestyle='--', linewidth=2, label='Sin efecto')
ax2.set_yticks(y_pos)
ax2.set_yticklabels(egma_nombres, fontsize=10)
ax2.set_xlabel('ATE (puntos porcentuales)', fontsize=11)
ax2.set_title('Forest Plot - Subpruebas EGMA (Matemáticas)\nIC 95% Wild Cluster Bootstrap', fontweight='bold', fontsize=12)
ax2.grid(True, alpha=0.3, axis='x')
ax2.legend(loc='lower right')

# Agregar p-valores
for i, (ate, p) in enumerate(zip(egma_ates, egma_pvals)):
    sig = "***" if p < 0.01 else "**" if p < 0.05 else "*" if p < 0.10 else ""
    ax2.annotate(f'p={p:.3f}{sig}', xy=(ate, i), xytext=(5, 0),
                 textcoords='offset points', fontsize=9, va='center')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'Forest_Plot_Subpruebas.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"\n[OK] Forest_Plot_Subpruebas.png guardado en {OUTPUT_DIR}")

# 13. TABLA RESUMEN DE RESULTADOS

In [ ]:
# ==============================================================================
# TABLA RESUMEN
# ==============================================================================

print("\n" + "="*90)
print("TABLA RESUMEN DE RESULTADOS")
print("="*90)

print(f"\n{'Outcome':<15} {'ATE':>12} {'SE':>10} {'IC 95%':>28} {'p-valor':>12} {'Sig':>6}")
print("-" * 90)

for nombre, res in resultados.items():
    sig = "***" if res['p_value'] < 0.01 else "**" if res['p_value'] < 0.05 else "*" if res['p_value'] < 0.10 else ""
    ic = f"[{res['ic_lower']:.2f}, {res['ic_upper']:.2f}]"
    print(f"{nombre:<15} {res['ate']:>12.4f} {res['se']:>10.4f} {ic:>28} {res['p_value']:>12.4f} {sig:>6}")

print("-" * 90)
print(f"Significancia: *** p<0.01, ** p<0.05, * p<0.10")
print(f"Clusters: {df['codigo_dane'].nunique()} escuelas | N: {len(df)} estudiantes")

# 14. VISUALIZACIONES FINALES

In [ ]:
# ==============================================================================
# FIGURA 1: ANÁLISIS EXPLORATORIO
# ==============================================================================

fig1, axes = plt.subplots(2, 2, figsize=(14, 10))

for idx, (outcome, ax_row) in enumerate(zip(['EGRA_Total', 'EGMA_Total'], [0, 1])):
    # Histogramas
    ax = axes[ax_row, 0]
    for trat, color, label in [(1, 'steelblue', 'Tratamiento'), (0, 'coral', 'Control')]:
        data = df[df['tratamiento'] == trat][outcome]
        ax.hist(data, bins=30, alpha=0.6, color=color, label=f'{label} (n={len(data)})', density=True)
    ax.set_xlabel(f'Puntaje {outcome.split("_")[0]}')
    ax.set_ylabel('Densidad')
    ax.set_title(f'Distribución {outcome.split("_")[0]} por Grupo', fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Boxplot por escuela
    ax2 = axes[ax_row, 1]
    escuelas_orden = df.groupby('codigo_dane')[outcome].median().sort_values().index
    colores = ['steelblue' if df[df['codigo_dane']==e]['tratamiento'].iloc[0]==1 else 'coral' for e in escuelas_orden]
    bp = ax2.boxplot([df[df['codigo_dane']==e][outcome].values for e in escuelas_orden],
                      positions=range(len(escuelas_orden)), patch_artist=True)
    for patch, color in zip(bp['boxes'], colores):
        patch.set_facecolor(color)
        patch.set_alpha(0.6)
    ax2.set_xlabel('Escuelas')
    ax2.set_ylabel(f'Puntaje {outcome.split("_")[0]}')
    ax2.set_title(f'{outcome.split("_")[0]} por Escuela (Azul=Trat, Rojo=Ctrl)', fontweight='bold')
    ax2.set_xticks([])
    ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'Analisis_Exploratorio.png', dpi=150, bbox_inches='tight')
plt.show()
print("[OK] Analisis_Exploratorio.png")

In [ ]:
# ==============================================================================
# FIGURA 2: DIAGNÓSTICOS DEL MODELO
# ==============================================================================

fig2, axes = plt.subplots(2, 2, figsize=(14, 10))

# ROC
ax1 = axes[0, 0]
fpr, tpr, _ = roc_curve(y, ps)
ax1.plot(fpr, tpr, 'b-', linewidth=2, label=f'ROC (AUC = {auroc:.3f})')
ax1.plot([0, 1], [0, 1], 'k--', alpha=0.5)
ax1.fill_between(fpr, tpr, alpha=0.3)
ax1.set_xlabel('FPR')
ax1.set_ylabel('TPR')
ax1.set_title('Curva ROC', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# PS distribution
ax2 = axes[0, 1]
ax2.hist(ps[y==1], bins=20, alpha=0.6, label='Tratamiento', density=True, color='steelblue')
ax2.hist(ps[y==0], bins=20, alpha=0.6, label='Control', density=True, color='coral')
ax2.set_xlabel('Propensity Score')
ax2.set_ylabel('Densidad')
ax2.set_title('Distribución de Propensity Scores', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Love Plot
ax3 = axes[1, 0]
smds_sorted = sorted(smds_crudo.items(), key=lambda x: x[1], reverse=True)
vars_s = [x[0] for x in smds_sorted]
vals_s = [x[1] for x in smds_sorted]
colors_s = ['red' if v > 0.25 else 'orange' if v > 0.10 else 'green' for v in vals_s]
ax3.barh(vars_s, vals_s, color=colors_s, alpha=0.7)
ax3.axvline(0.10, color='green', linestyle='--', linewidth=2)
ax3.axvline(0.25, color='red', linestyle='--', linewidth=2)
ax3.set_xlabel('SMD')
ax3.set_title('Balance de Covariables', fontweight='bold')
ax3.grid(True, alpha=0.3, axis='x')

# PS boxplot
ax4 = axes[1, 1]
bp = ax4.boxplot([ps[y==0], ps[y==1]], labels=['Control', 'Tratamiento'], patch_artist=True)
bp['boxes'][0].set_facecolor('coral')
bp['boxes'][1].set_facecolor('steelblue')
ax4.set_ylabel('Propensity Score')
ax4.set_title('PS por Grupo', fontweight='bold')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'Diagnosticos_modelo.png', dpi=150, bbox_inches='tight')
plt.show()
print("[OK] Diagnosticos_modelo.png")

In [ ]:
# ==============================================================================
# FIGURA 3: RESULTADOS FINALES
# ==============================================================================

fig3, axes = plt.subplots(2, 2, figsize=(14, 10))

# Forest Plot
ax1 = axes[0, 0]
nombres = list(resultados.keys())
ates = [r['ate'] for r in resultados.values()]
lowers = [r['ic_lower'] for r in resultados.values()]
uppers = [r['ic_upper'] for r in resultados.values()]
y_pos = np.arange(len(nombres))
errors = [[ate - l for ate, l in zip(ates, lowers)], [u - ate for ate, u in zip(ates, uppers)]]
colors = ['gray' for _ in nombres]
ax1.barh(y_pos, ates, xerr=errors, color=colors, alpha=0.7, capsize=5, edgecolor='black')
ax1.axvline(0, color='black', linestyle='-', linewidth=1)
ax1.set_yticks(y_pos)
ax1.set_yticklabels(nombres)
ax1.set_xlabel('ATE (puntos)')
ax1.set_title('Forest Plot - Efectos del Tratamiento (IC 95%)', fontweight='bold')
ax1.grid(True, alpha=0.3, axis='x')

# Bootstrap EGRA
ax2 = axes[0, 1]
ax2.hist(resultados['EGRA Total']['bootstrap_dist'], bins=50, alpha=0.7, color='steelblue', edgecolor='black')
ax2.axvline(resultados['EGRA Total']['ate'], color='red', linestyle='-', linewidth=2, label='ATE')
ax2.axvline(0, color='black', linestyle='--', linewidth=1.5, label='Efecto nulo')
ax2.set_xlabel('ATE')
ax2.set_ylabel('Frecuencia')
ax2.set_title('Bootstrap - EGRA Total', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Bootstrap EGMA
ax3 = axes[1, 0]
ax3.hist(resultados['EGMA Total']['bootstrap_dist'], bins=50, alpha=0.7, color='forestgreen', edgecolor='black')
ax3.axvline(resultados['EGMA Total']['ate'], color='red', linestyle='-', linewidth=2, label='ATE')
ax3.axvline(0, color='black', linestyle='--', linewidth=1.5, label='Efecto nulo')
ax3.set_xlabel('ATE')
ax3.set_ylabel('Frecuencia')
ax3.set_title('Bootstrap - EGMA Total', fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Comparación
ax4 = axes[1, 1]
diff_simples = [df[df['tratamiento']==1][o].mean() - df[df['tratamiento']==0][o].mean() for o in ['EGRA_Total', 'EGMA_Total']]
aipw_ates = [r['ate'] for r in resultados.values()]
x = np.arange(2)
width = 0.35
ax4.bar(x - width/2, diff_simples, width, label='Diferencia Simple', color='lightblue', edgecolor='black')
ax4.bar(x + width/2, aipw_ates, width, label='AIPW/DR', color='steelblue', edgecolor='black')
ax4.axhline(0, color='black', linestyle='-', linewidth=0.5)
ax4.set_xticks(x)
ax4.set_xticklabels(['EGRA', 'EGMA'])
ax4.set_ylabel('Efecto')
ax4.set_title('Diferencia Simple vs AIPW/DR', fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'Resultados_finales.png', dpi=150, bbox_inches='tight')
plt.show()
print("[OK] Resultados_finales.png")

# 15. GUARDAR RESULTADOS

In [ ]:
# ==============================================================================
# GUARDAR RESULTADOS
# ==============================================================================

print("\n" + "="*80)
print("GUARDANDO RESULTADOS")
print("="*80)

resultados_df = pd.DataFrame([
    {
        'Outcome': nombre,
        'ATE': res['ate'],
        'SE': res['se'],
        'IC_Lower': res['ic_lower'],
        'IC_Upper': res['ic_upper'],
        'p_value': res['p_value'],
        't_stat': res['t_stat'],
        'Significativo_05': res['p_value'] < 0.05,
        'Significativo_10': res['p_value'] < 0.10
    }
    for nombre, res in resultados.items()
])

resultados_df.to_csv(OUTPUT_DIR / 'resultados_finales.csv', index=False, encoding='utf-8-sig')
print(f"\n[OK] resultados_finales.csv")
display(resultados_df)

diagnosticos = {
    'AUROC': auroc,
    'SMD_promedio': smd_promedio_crudo,
    'ESS_tratamiento_pct': ess_t / n_tratados * 100,
    'ESS_control_pct': ess_c / n_control * 100,
    'N_estudiantes': len(df),
    'N_escuelas': df['codigo_dane'].nunique(),
    'N_bootstrap': CONFIG['N_BOOTSTRAP']
}

pd.DataFrame([diagnosticos]).to_csv(OUTPUT_DIR / 'diagnosticos.csv', index=False)
print(f"[OK] diagnosticos.csv")

# 16. CONCLUSIONES FINALES

---

## Hallazgo Principal

**No se encontraron efectos estadísticamente significativos del programa PTA/FI 3.0 sobre los resultados de EGRA (lectura) ni EGMA (matemáticas).**

| Outcome | ATE | IC 95% | p-valor | Significativo |
|---------|-----|--------|---------|---------------|
| EGRA Total | -1.41 | [-6.84, 3.72] | 0.601 | No |
| EGMA Total | -1.74 | [-6.41, 2.83] | 0.467 | No |

---

## Aspectos Metodológicos

- **Estimador AIPW/DR**: Doblemente robusto
- **Wild Cluster Bootstrap (B=1000)**: Inferencia válida con pocos clusters
- **Cross-Fitting con GroupKFold**: Evita sobreajuste
- **Gradient Boosting**: Captura relaciones no lineales

## Diagnósticos

- **AUROC**: 0.803 (rango óptimo)
- **ESS**: >85% (poca pérdida de información)
- **N**: 770 estudiantes, 22 escuelas

---

In [ ]:
# ==============================================================================
# RESUMEN FINAL
# ==============================================================================

print("\n" + "="*80)
print("ANÁLISIS COMPLETADO EXITOSAMENTE")
print("="*80)

print(f"""
RESUMEN DE LA EVALUACIÓN DE IMPACTO PTA/FI 3.0
{'='*60}

MUESTRA:
  - Estudiantes: {len(df):,}
  - Escuelas: {df['codigo_dane'].nunique()}

DIAGNÓSTICOS:
  - AUROC: {auroc:.3f}
  - SMD Promedio: {smd_promedio_crudo:.3f}

RESULTADOS:
  EGRA Total: ATE = {resultados['EGRA Total']['ate']:.2f} (p={resultados['EGRA Total']['p_value']:.3f})
  EGMA Total: ATE = {resultados['EGMA Total']['ate']:.2f} (p={resultados['EGMA Total']['p_value']:.3f})

CONCLUSIÓN:
  No se encontraron efectos estadísticamente significativos.

{'='*60}
""")